In [1]:
import os
import sys
import urllib.parse
import urllib.request as req
import json
import re
import time
import datetime
import pandas as pd
import pymysql

In [2]:
client_id = "3aby98TxbX87ykaP5tKI"
client_secret = "j5oBzNi350"

In [3]:
def save_db(tuple_result):
    conn = pymysql.Connect(
        host="localhost",
        user="jhm",
        password="1234",
        database="pydb",
        charset="utf8"
    )
    cursor = conn.cursor()
    cursor.executemany(
            "INSERT INTO naver_book VALUES(%s, %s, %s, %s, %s)",
            tuple_result
        )
    conn.commit()
    conn.close()

In [4]:
def save_csv(tuple_result, text, node):
    dataframe = pd.DataFrame(tuple_result, columns=["cnt", "title", "description", "link", "pubDate"])
    dataframe.to_csv(f"../data/{text}_naver_{node}.csv", index=False, encoding="utf-8")

In [5]:
def get_post_data(item, json_result, tuple_result, cnt):
    title = item["title"].replace("<b>", "").replace("</b>", "").replace("&quot;", "")
    description = item["description"].replace("<b>", "").replace("</b>", "").replace("&quot;", "")
    link = item["link"]
    pubDate = datetime.datetime.strptime(item["pubDate"], "%a, %d %b %Y %H:%M:%S +0900").strftime("%Y-%m-%d %H:%M:%S")  # 포매팅을 거쳐서 String으로 만들었음
    tuple_result.append((cnt, title, description, link, pubDate))   # 튜플 리스트로 만드는 방식
    json_result.append({"cnt": cnt, "title": title, "description": description, "link": link, "pubDate": pubDate})  # JSON(딕셔너리) 형으로 만드는 방식

In [6]:
def get_request_url(url):
    request = req.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    try:
        response = req.urlopen(request)
        if response.getcode() == 200:
            print(f"{datetime.datetime.now()} URL 요청 성공")
            return response.read().decode("utf-8")
        else:
            print(response.getcode())
    except Exception as e:
        print(e)
        print(f"{datetime.datetime.now()} 요청 실패: {url}")

In [7]:
def get_naver_search(node, src_text, start, display):
    url = f"https://openapi.naver.com/v1/search/{node}?query={src_text}&start={start}&display={display}"
    res_decode = get_request_url(url)
    if res_decode == None:
        return None
    else:
        return json.loads(res_decode)

In [8]:
def news_search():
    node = "news"
    text = input("검색어를 입력하세요.")
    src_text = urllib.parse.quote(text)
    cnt = 0
    tuple_result = []
    json_result = []
    json_response = get_naver_search(node, src_text, 1, 100)    # 노드, 검색어, start, display
    while (json_response != None) and (json_response["display"] != 0):
        for item in json_response["items"]:
            cnt += 1
            get_post_data(item, json_result, tuple_result, cnt)
        start = json_response["start"] + json_response["display"]
        json_response = get_naver_search(node, src_text, start, 100)
    with open(f"../data/{text}_naver_{node}.json", "w", encoding="utf-8") as f:
        jsonFile = json.dumps(json_result, indent=2, sort_keys=False, ensure_ascii=False)
            # 들여쓰기 2를 주어 보기좋게 만들고(기본값 False), key를 자동으로 오름차순 정렬하지 않고(기본값 False), 아스키코드로 변환 안한다는 뜻(기본값 True)
        f.write(jsonFile)
    save_db(tuple_result)
    save_csv(tuple_result, text, node)


In [9]:
news_search()

HTTP Error 400: Bad Request
2025-03-25 15:41:38.793981 요청 실패: https://openapi.naver.com/v1/search/news?query=&start=1&display=100


요기까지가 메소드로 뉴스 서치하기 예제

In [10]:
# 네이버 검색 API 예제 - 블로그 검색
encText = urllib.parse.quote("손흥민") # 한글을 16진수로 파싱
# url = "https://openapi.naver.com/v1/search/image?query=" + encText # JSON 결과(이미지)
# url = "https://openapi.naver.com/v1/search/book?query=" + encText # JSON 결과(도서)
url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과(뉴스)
# url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과(블로그)
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과(블로그)
request = urllib.request.Request(url)   # request 객체 생성
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode == 200):
    response_body = response.read()
    result = response_body.decode('utf-8')    # 지금 상황에선 아직 JSON 형태가 아님... 그냥 String 형태임
    result_json = json.loads(result)   # 이제 JSON임
    items = result_json["items"]
    item_list = []
    for item in items:
        title = item["title"].replace("&quot;", '').replace("<b>", "").replace("</b>", "").replace("&lt;", "").replace("&gt;", "")
        link = item["link"]
        pubDate = item["pubDate"]
        description = item["description"].replace("&quot;", '').replace("<b>", "").replace("</b>", "")
        item_list.append((title, link, pubDate, description))
else:
    print("Error Code:" + rescode)

df = pd.DataFrame(item_list)
df

,0,1,2,3
0,손흥민 어떻게 이런 감독 밑에서 일했나...쓰레기 같은 기분 느껴 무시...,https://m.sports.naver.com/wfootball/article/0...,"Tue, 25 Mar 2025 15:35:00 +0900",안토니오 콘테 감독은 특정 선수들을 완전히 무시해버렸다. 영국 이브닝 스탠다드는 2...
1,"[스포츠JOB아보기(20) 정성은] '진로의 연결고리' 대외활동, 합격 꿀팁은...",http://www.sportsq.co.kr/news/articleView.html...,"Tue, 25 Mar 2025 15:34:00 +0900","손흥민을 보고 체육대학 진학을 결심한 이후 축구산업, 마케팅, 사업계획 등 여러 경..."
2,로메로 ‘손흥민급’ 주급 제안에도 토트넘 의료진 저격…레알 마드리...,https://m.sports.naver.com/wfootball/article/1...,"Tue, 25 Mar 2025 15:28:00 +0900",토트넘은 로메로를 잡기 위해 주장 손흥민과 동급인 주급 20만파운드(약 3억8000...
3,[IN POINT] 클린스만호 멸망→한국 축구 혼란 만든 요르단...이제 홍명보...,https://m.sports.naver.com/kfootball/article/4...,"Tue, 25 Mar 2025 15:25:00 +0900","이전부터 클린스만 감독 아래 쌓였던 불만과 비판이 터졌고 손흥민, 이강인 불화 문제..."
4,쿠팡플레이가 EPL 전 경기를 독점 생중계한다,https://www.esquirekorea.co.kr/article/1879150,"Tue, 25 Mar 2025 15:20:00 +0900",손흥민(@hm_son7) 쿠팡플레이가 2025-2026 시즌부터 프리미어리그 전 경...
5,"손흥민과 적으로 만날 수도? 케인, EPL 복귀 관심...리버풀은 1,900억 제...",https://m.sports.naver.com/wfootball/article/4...,"Tue, 25 Mar 2025 15:15:00 +0900",손흥민과 적으로 다시 만난다? 영국 매체 '90min'은 25일 리버풀은 해리 케인...
6,한국 요르단 축구 중계… 선발 라인업은?,https://www.wikitree.co.kr/articles/1036174,"Tue, 25 Mar 2025 15:00:00 +0900","(버밍엄시티), 손흥민(토트넘), 양민혁(퀸즈파크레인저스), 양현준(셀틱), 엄지성..."
7,'대충격!' 손흥민 팔면 토트넘 대반전 가능!! 英 매체 충격적 분석......,https://m.sports.naver.com/wfootball/article/1...,"Tue, 25 Mar 2025 14:56:00 +0900",손흥민(토트넘)에 대한 평가가 최악으로 치닫고 있다. 더이상 좋은 모습을 보이지 못...
8,"케인의 토트넘-손흥민 배신 시작됐나...리버풀, 1900억 입찰! PL 복귀 추...",https://m.sports.naver.com/wfootball/article/4...,"Tue, 25 Mar 2025 14:50:00 +0900","손흥민, 델레 알리, 크리스티안 에릭센과 함께 'DESK 라인'을 구성하면서 토트넘..."
9,"토트넘 유망주들, 반드시 꺾어야 할 '적과의 동침' 후 복귀… 친구이자...",https://m.sports.naver.com/wfootball/article/4...,"Tue, 25 Mar 2025 14:48:00 +0900",토트넘 윙어 중 가장 능숙한 드리블러로서 손흥민과 좌우를 맡았을 때 조합이 좋은 편...


book 검색
제목, 저자, 출판사, 가격, 이미지 링크, 출판일
200건 검색

In [11]:
item_list = []

search_list = ["AI", "파이썬", "한국사"]

for search in search_list:
    for i in range(1, 11):
        encText = urllib.parse.quote(search)
        url = "https://openapi.naver.com/v1/search/book?start=" + str(i) + "&query=" + encText
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode == 200):
            response_body = response.read()
            result = json.loads(response_body.decode('utf-8'))
            items = result["items"]
            for item in items:
                title = item["title"]
                image = item["image"]
                author = item["author"]
                discount = item["discount"]
                publisher = item["publisher"]
                pubdate = item["pubdate"]
                item_list.append((title, author, publisher, discount, image, pubdate))
        else:
            print("Error Code:" + rescode)
            break

In [ ]:
df = pd.DataFrame(item_list, columns=["제목", "저자", "출판사", "가격", "이미지 링크", "출판일"])
df.to_csv("../data/네이버 도서 검색 예제.csv", encoding="utf-8")

In [ ]:
# 네이버 데이터랩(검색어트렌드) 사용 예제
#-*- coding: utf-8 -*-

client_id = "tVgSoOb5eAZXCc9FtW37"
client_secret = "WUaCR5ONEY"

url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\"2025-02-01\",\"endDate\":\"2025-03-01\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"검색어\",\"keywords\":[\"손흥민\"]}]}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
    
else:
    print("Error Code:" + rescode)

{"startDate":"2025-02-01","endDate":"2025-03-01","timeUnit":"date","results":[{"title":"검색어","keywords":["손흥민"],"data":[{"period":"2025-02-01","ratio":27.99637},{"period":"2025-02-02","ratio":41.7926},{"period":"2025-02-03","ratio":70.94084},{"period":"2025-02-04","ratio":28.57537},{"period":"2025-02-05","ratio":26.67557},{"period":"2025-02-06","ratio":36.18687},{"period":"2025-02-07","ratio":54.16973},{"period":"2025-02-08","ratio":25.94606},{"period":"2025-02-09","ratio":27.35286},{"period":"2025-02-10","ratio":40.63613},{"period":"2025-02-11","ratio":39.55031},{"period":"2025-02-12","ratio":36.43721},{"period":"2025-02-13","ratio":34.53741},{"period":"2025-02-14","ratio":29.67041},{"period":"2025-02-15","ratio":27.24996},{"period":"2025-02-16","ratio":31.06186},{"period":"2025-02-17","ratio":55.29702},{"period":"2025-02-18","ratio":27.90883},{"period":"2025-02-19","ratio":24.00939},{"period":"2025-02-20","ratio":28.02555},{"period":"2025-02-21","ratio":44.01492},{"period":"2025-02-2